In [1]:
import os
os.chdir("./../.")

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import json
import dotenv

In [4]:
dotenv.load_dotenv()

True

In [5]:

from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [6]:
from src.utils_stock import get_top_nasdaq_stock_data

In [7]:
# parameter
NUM_DAYS = 5

In [8]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [9]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

## Define tool

In [10]:
@tool
def get_top_nasdaq_performance_stock_data() -> str:
    """Get the day's top NASDAQ-100 gainer with symbol, percentage increase, sample data in markdown format, pandas dataframe info and csv data path for further python code analysis.
    """
    result = get_top_nasdaq_stock_data(NUM_DAYS)
    return json.dumps(result)

### Define Agent

In [11]:
# Research agent and node
data_acquisition_agent = create_react_agent(
    llm,
    tools=[get_top_nasdaq_performance_stock_data],
    prompt=get_system_prompt(
        "You can only do data acquisition to the day's top NASDAQ gainer info using get_top_nasdaq_performance_stock_data tool."
    ),
)

### Define Graph

In [12]:


from typing import Literal
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, BaseMessage

In [13]:
def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto

In [14]:
def data_acquisition_node(
    state: MessagesState,
) -> Command[Literal[END]]:  # type: ignore
    result = data_acquisition_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], END)
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="data_acquisition_specialist"
    )
    return Command(
        update={
            # share internal message history of research agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )

In [15]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(MessagesState)
workflow.add_node("data_acquisition_specialist", data_acquisition_node)

workflow.add_edge(START, "data_acquisition_specialist")
graph = workflow.compile()

In [16]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [18]:
events = graph.stream(
    {
        "messages": [
            (
                "user",
                "First, get the top performance stock data "
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 10},
)
for s in events:
    print(s)
    print("----")

100%|██████████| 101/101 [00:02<00:00, 34.68it/s]


{'data_acquisition_specialist': {'messages': [HumanMessage(content='First, get the top performance stock data ', additional_kwargs={}, response_metadata={}, id='9833673c-7550-4a66-ac91-602f0fb25af5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_4208df73-e073-416f-a4ae-867f649af681', 'function': {'arguments': '{}', 'name': 'get_top_nasdaq_performance_stock_data'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 237, 'total_tokens': 260, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'df59e8c5-3d2e-4414-9080-53f01fdeebf0', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--88219a56-8826-49c1-95b0-8388232018bb-0', tool_calls=[{'name': 'get_top_n